In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from gensim.models import Word2Vec

In [2]:
# DBN architecture
class DBN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DBN, self).__init__()
        self.rbm1 = RBM(input_dim, hidden_dim)
        self.rbm2 = RBM(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = torch.sigmoid(self.rbm1(x))
        x = torch.sigmoid(self.rbm2(x))
        x = self.fc(x)
        return x

# RBM layer
class RBM(nn.Module):
    def __init__(self, visible_dim, hidden_dim):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(visible_dim, hidden_dim))
        self.visible_bias = nn.Parameter(torch.randn(visible_dim))
        self.hidden_bias = nn.Parameter(torch.randn(hidden_dim))
    
    def forward(self, x):
        p_hidden_given_visible = torch.sigmoid(torch.matmul(x, self.W) + self.hidden_bias)
        sampled_hidden = torch.bernoulli(p_hidden_given_visible)
        p_visible_given_hidden = torch.sigmoid(torch.matmul(sampled_hidden, self.W.t()) + self.visible_bias)
        return p_visible_given_hidden

In [4]:
# Connect to Neo4j and retrieve knowledge graph vectors
class Neo4jDataLoader:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def get_vectors(self):
        with self._driver.session() as session:
            # query = "MATCH (node:Entity) RETURN node.vector AS vector"
            query = "MATCH (n) RETURN n.Vector AS vector"
            result = session.run(query)
            vectors = [record['vector'] for record in result]
        return torch.tensor(vectors)
neo4j_loader = Neo4jDataLoader(uri="bolt://34.101.159.175:7687", user="neo4j", password="unej1234")
data = neo4j_loader.get_vectors()


In [7]:
uri = "bolt://34.101.159.175:7687"
username = "neo4j"
password = "unej1234"

driver = GraphDatabase.driver(uri, auth=(username, password))
with driver.session() as session:
    result = session.run("MATCH (n) RETURN n.label AS label")
    label = pd.DataFrame([record.values() for record in result], columns=result.keys())
    
# Extract the values from the 'label' column and convert to a one-dimensional list
label_values = label['label'].tolist()

In [8]:
data

tensor([[-0.4801,  0.6460, -0.9932,  ..., -0.4361, -0.8677, -1.7822],
        [ 0.2617,  0.0716,  0.0499,  ..., -1.2279, -0.1212, -1.0645],
        [ 0.0228,  0.3414, -0.4033,  ..., -0.0076, -0.9350, -1.0907],
        ...,
        [-0.4614,  0.1860,  0.3704,  ..., -1.3313, -0.4901, -0.2599],
        [-0.2461,  0.1481, -0.3852,  ..., -0.1658, -0.8010, -1.0049],
        [ 1.8812,  0.1237, -2.4698,  ..., -1.5062, -2.7816,  0.4476]])

In [9]:
# Define a Word2Vec model (you need to define this as you did before)
sentences = [str(text).split() for text in label_values]
model = Word2Vec(sentences, vector_size=1, window=5, min_count=1, sg=0)

# Function to get embeddings for a list of words
def get_sentence_embedding(word_list):
    word_vectors = [model.wv[word] for word in word_list if word in model.wv.key_to_index]
    
    if word_vectors:
        sentence_embedding = sum(word_vectors)
        return sentence_embedding
    else:
        return None

# Sample DataFrame
data = {'text_data': label_values}
df = pd.DataFrame(data)

# Split the text_data column into lists of words and apply the function to each row
df['text_data'] = df['text_data'].apply(lambda x: x.split() if x is not None else [])
df['embeddings'] = df['text_data'].apply(lambda x: get_sentence_embedding(x) if x else None)

print(df['embeddings'])

# # Filter out rows where embeddings are not available
# df = df.dropna(subset=['embeddings'])

# Replace rows where embeddings are not available with a default value (e.g., zeros)
default_embedding = np.zeros(1)  # Replace with your desired default value
df['embeddings'] = df['embeddings'].apply(lambda x: x if x is not None else default_embedding)

# Convert embeddings to a PyTorch tensor
embeddings_tensor = torch.tensor(df['embeddings'].to_list())

print(embeddings_tensor)


0        [0.78980684]
1         [1.4231793]
2         [0.9084352]
3       [-0.38928694]
4       [-0.74302006]
            ...      
100       [0.6022621]
101      [0.13106883]
102      [-1.4819928]
103      [0.06058532]
104    [-0.056747735]
Name: embeddings, Length: 105, dtype: object
tensor([[ 0.7898],
        [ 1.4232],
        [ 0.9084],
        [-0.3893],
        [-0.7430],
        [ 0.4056],
        [-0.4913],
        [-0.6787],
        [-0.5073],
        [ 0.9615],
        [ 0.9742],
        [-2.0069],
        [-0.1508],
        [ 0.1580],
        [-0.7407],
        [ 0.5534],
        [-0.2743],
        [ 0.2260],
        [-0.5307],
        [ 1.1660],
        [ 0.8677],
        [ 0.5838],
        [-0.7661],
        [ 0.4820],
        [-0.8932],
        [-0.1494],
        [ 0.6777],
        [-0.3936],
        [-0.1938],
        [ 0.5743],
        [ 0.0045],
        [ 1.0079],
        [-0.9604],
        [ 0.5007],
        [-0.8760],
        [-0.7847],
        [-0.9132],
        [-

/var/folders/tq/8dxrnbkd5bx737ykn8j5w4j40000gn/T/ipykernel_2882/1662484291.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  embeddings_tensor = torch.tensor(df['embeddings'].to_list())


In [10]:
embeddings_tensor.shape

torch.Size([105, 1])

In [13]:
# Main training loop
def train_dbn():
    # Initialize DBN and other hyperparameters
    input_dim = 32  # Adjust based on the dimensionality of your knowledge graph vectors
    hidden_dim = 32
    output_dim = 2  # Adjust based on your task (e.g., classification)

    dbn = DBN(input_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(dbn.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Connect to Neo4j and retrieve data
    neo4j_loader = Neo4jDataLoader(uri="bolt://34.101.159.175:7687", user="neo4j", password="unej1234")
    data = neo4j_loader.get_vectors()

    # Load labels for your data
    labels = embeddings_tensor

    # Create a DataLoader to handle batching (if needed)
    batch_size = 32  # Adjust based on your dataset size
    dataset = TensorDataset(data, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    for epoch in range(100):  # Adjust the number of epochs
        total_loss = 0.0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            output = dbn(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/100], Loss: {total_loss / len(dataloader)}")

if __name__ == "__main__":
    train_dbn()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x10 and 32x32)